In [4]:
import os

# SMPL 모델 파일이 있는지 확인
SMPL_DIR = r"C:\Users\SongYoengEun\Desktop\cap\dataset\aist_plusplus_final"

smpl_male = os.path.join(SMPL_DIR, "SMPL_MALE.pkl")
smpl_female = os.path.join(SMPL_DIR, "SMPL_FEMALE.pkl")

print(f"SMPL_MALE.pkl 존재: {os.path.exists(smpl_male)}")
print(f"SMPL_FEMALE.pkl 존재: {os.path.exists(smpl_female)}")

# motions 폴더 확인
MOTION_DIR = os.path.join(SMPL_DIR, "motions")
pkl_files = [f for f in os.listdir(MOTION_DIR) if f.endswith('.pkl')]
print(f"\nMotion 파일 개수: {len(pkl_files)}")
print(f"첫 번째 파일: {pkl_files[0] if pkl_files else 'None'}")

SMPL_MALE.pkl 존재: True
SMPL_FEMALE.pkl 존재: True

Motion 파일 개수: 1408
첫 번째 파일: gBR_sBM_cAll_d04_mBR0_ch01.pkl


In [8]:
import pickle
import numpy as np
import cv2
from tqdm.notebook import tqdm

# 출력 폴더 생성
OUTPUT_DIR = r"C:\Users\SongYoengEun\Desktop\cap\dance_video"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 테스트용 파일 하나 선택
test_file = "gBR_sBM_cAll_d04_mBR0_ch01.pkl"

# 모션 데이터 로드
with open(os.path.join(MOTION_DIR, test_file), 'rb') as f:
    data = pickle.load(f)

print("데이터 구조:")
print(f"  smpl_poses: {data['smpl_poses'].shape}")
print(f"  smpl_trans: {data['smpl_trans'].shape}")
print(f"  frame_count: {data['smpl_trans'].shape[0]}")

데이터 구조:
  smpl_poses: (720, 72)
  smpl_trans: (720, 3)
  frame_count: 720


In [10]:
import torch
from smplx import SMPL

# GPU 사용 가능하면 GPU, 아니면 CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# SMPL 모델 로드
smpl = SMPL(
    model_path=SMPL_DIR,
    gender='MALE',
    batch_size=1
).to(device)

print("✓ SMPL 모델 로드 완료")

Using device: cpu
✓ SMPL 모델 로드 완료


In [11]:
import torch
from smplx import SMPL

# 2단계에서 사용했던 경로를 여기에 직접 정의합니다.
SMPL_DIR = r"C:\Users\SongYoengEun\Desktop\cap\dataset\aist_plusplus_final" 

# GPU 사용 가능하면 GPU, 아니면 CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# SMPL 모델 로드
smpl = SMPL(
    model_path=SMPL_DIR,
    gender='MALE',
    batch_size=1
).to(device)

print("✓ SMPL 모델 로드 완료")

Using device: cpu
✓ SMPL 모델 로드 완료


In [34]:
import torch
import cv2
import pickle
import numpy as np
import os
import trimesh
import pyrender
from tqdm.notebook import tqdm

# 주의: smpl, device, SMPL_DIR, MOTION_DIR, OUTPUT_DIR 변수가 이전 셀에서 정의되어 있어야 합니다!

# -------------------------------------------------------------
# 3D 렌더링을 위한 설정값 함수
# -------------------------------------------------------------
def get_render_settings(width=1920, height=1080):
    # Pyrender 렌더러 설정
    
    render_width, render_height = width, height 

    # 카메라 설정
    # 필드 오브 뷰(fov)를 조정하여 모델이 화면에 들어오는 정도를 조절할 수 있습니다.
    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=render_width/render_height)
    
    # 조명 설정 (밝은 조명)
    light = pyrender.DirectionalLight(color=np.ones(3), intensity=2.0)
    
    # 장면 설정 (흰색 배경)
    scene = pyrender.Scene(bg_color=np.array([1.0, 1.0, 1.0])) 
    
    return scene, camera, light, render_width, render_height


def render_smpl_video_mesh(motion_file, output_dir=OUTPUT_DIR):
    """SMPL 메쉬를 3D 렌더러(Pyrender)를 사용하여 영상으로 렌더링"""
    
    global smpl, device, MOTION_DIR
    
    # 모션 데이터 로드
    motion_path = os.path.join(MOTION_DIR, motion_file)
    with open(motion_path, 'rb') as f:
        data = pickle.load(f)
        
    # 데이터 준비
    poses = torch.tensor(
        data['smpl_poses'].reshape(-1, 24, 3),
        dtype=torch.float32
    ).to(device)
    
    trans = torch.tensor(
        data['smpl_trans'],
        dtype=torch.float32
    ).to(device)
    
    frame_count = poses.shape[0]
    print(f"Total frames: {frame_count}")
    
    # 렌더링 설정 초기화
    scene, camera, light, width, height = get_render_settings()
    
    # 렌더러 초기화
    r = pyrender.OffscreenRenderer(width, height)
    
    # Scene에 Light와 Camera를 추가합니다.
    scene.add(light, pose=np.eye(4))
    camera_node = scene.add(camera, pose=np.eye(4))

    # 초기 메쉬 노드 변수만 선언 (루프에서 제거/추가할 예정)
    mesh_node = None 
    
    # 비디오 설정
    fps = 60
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # 파일명 접미사 변경: _mesh.mp4 -> _body.mp4
    output_path = os.path.join(output_dir, motion_file.replace('.pkl', '_body.mp4')) 
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # -------------------------------------------------------------
    # 프레임별 렌더링 루프
    # -------------------------------------------------------------
    for i in tqdm(range(frame_count), desc=f"Rendering {motion_file}"):
        
        # 1. SMPL Forward Pass (정점(Vertices) 가져오기)
        with torch.no_grad():
            output = smpl(
                body_pose=poses[i:i+1, 1:],
                global_orient=poses[i:i+1, :1],
                transl=trans[i:i+1]
            )
        
        vertices = output.vertices[0].cpu().numpy()  # (6890, 3)
        
        # X, Y축 오프셋 제거 및 중앙 고정
        # Y축 평균값을 계산할 때, 바닥에 더 가깝게 위치하도록 조정합니다.
        # 발의 가장 낮은 지점을 기준으로 평균을 조정하여 발이 잘리지 않게 합니다.
        center = vertices.mean(axis=0)
        vertices -= center 
        
        # 2. 메쉬 노드 업데이트 (이전 노드 제거 후 새 노드 추가)
        if mesh_node is not None:
            scene.remove_node(mesh_node)

        # 새로운 정점과 기존 면으로 메쉬 객체 생성
        new_mesh = trimesh.Trimesh(vertices, smpl.faces, process=False)
        pyrender_mesh = pyrender.Mesh.from_trimesh(new_mesh, smooth=True)
        
        # 장면(Scene)에 새 메쉬 노드를 추가하고 변수를 업데이트합니다.
        mesh_node = scene.add(pyrender_mesh)

        # 3. 카메라 설정 (메쉬를 바라보도록 위치 조정)
        camera_pose = np.eye(4)
        # 카메라 Z축 거리와 Y축 오프셋 조정
        camera_pose[2, 3] = 3.0  # Z축으로 3.0만큼 후퇴 (모델과의 거리) - 필요에 따라 2.5~3.5 사이로 조정
        camera_pose[1, 3] = -0.3 # Y축으로 -0.3만큼 (아래로) 이동하여 발이 잘리지 않도록 조정 (조정 필요)

        scene.set_pose(camera_node, camera_pose)
        
        # 4. 렌더링 및 저장
        color, _ = r.render(scene)
        
        # 5. OpenCV 형식으로 변환 및 저장
        frame = cv2.cvtColor(color, cv2.COLOR_RGB2BGR)
        
        # 프레임 번호 표시 제거 (주석 처리)
        # cv2.putText(
        #     frame, f"Frame: {i+1}/{frame_count}",
        #     (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2
        # )
        
        out.write(frame)
    
    r.delete() # 렌더러 리소스 해제
    out.release()
    print(f"✓ 완료: {output_path}")
    return output_path

In [36]:
video_path = render_smpl_video_mesh("gBR_sBM_cAll_d04_mBR0_ch01.pkl") 
print(f"생성된 비디오 경로: {video_path}")

Total frames: 720


Rendering gBR_sBM_cAll_d04_mBR0_ch01.pkl:   0%|          | 0/720 [00:00<?, ?it/s]

✓ 완료: C:\Users\SongYoengEun\Desktop\cap\dance_video\gBR_sBM_cAll_d04_mBR0_ch01_body.mp4
생성된 비디오 경로: C:\Users\SongYoengEun\Desktop\cap\dance_video\gBR_sBM_cAll_d04_mBR0_ch01_body.mp4


In [ ]:
# MOTION_DIR 내의 모든 .pkl 파일 목록 가져오기
pkl_files = [f for f in os.listdir(MOTION_DIR) if f.endswith('.pkl')]
print(f"총 {len(pkl_files)}개의 .pkl 파일을 찾았습니다.")

# 각 .pkl 파일에 대해 렌더링 함수 호출
for pkl_file in pkl_files:
    try:
        print(f"\n--- {pkl_file} 렌더링 시작 ---")
        render_smpl_video_mesh(pkl_file)
    except Exception as e:
        print(f" 오류 발생 ({pkl_file}): {e}")
        continue

print("\n✓ 모든 파일 처리 완료!")

총 1408개의 .pkl 파일을 찾았습니다.

--- gBR_sBM_cAll_d04_mBR0_ch01.pkl 렌더링 시작 ---
Total frames: 720


Rendering gBR_sBM_cAll_d04_mBR0_ch01.pkl:   0%|          | 0/720 [00:00<?, ?it/s]

✓ 완료: C:\Users\SongYoengEun\Desktop\cap\dance_video\gBR_sBM_cAll_d04_mBR0_ch01_body.mp4

--- gBR_sBM_cAll_d04_mBR0_ch02.pkl 렌더링 시작 ---
Total frames: 720


Rendering gBR_sBM_cAll_d04_mBR0_ch02.pkl:   0%|          | 0/720 [00:00<?, ?it/s]

✓ 완료: C:\Users\SongYoengEun\Desktop\cap\dance_video\gBR_sBM_cAll_d04_mBR0_ch02_body.mp4

--- gBR_sBM_cAll_d04_mBR0_ch03.pkl 렌더링 시작 ---
Total frames: 720


...

Rendering gBR_sBM_cAll_d05_mBR1_ch09.pkl:   0%|          | 0/640 [00:00<?, ?it/s]

✓ 완료: C:\Users\SongYoengEun\Desktop\cap\dance_video\gBR_sBM_cAll_d05_mBR1_ch09_body.mp4

--- gBR_sBM_cAll_d05_mBR1_ch10.pkl 렌더링 시작 ---
Total frames: 640


Rendering gBR_sBM_cAll_d05_mBR1_ch10.pkl:   0%|          | 0/640 [00:00<?, ?it/s]

✓ 완료: C:\Users\SongYoengEun\Desktop\cap\dance_video\gBR_sBM_cAll_d05_mBR1_ch10_body.mp4

--- gBR_sBM_cAll_d05_mBR4_ch01.pkl 렌더링 시작 ---
Total frames: 480


Rendering gBR_sBM_cAll_d05_mBR4_ch01.pkl:   0%|          | 0/480 [00:00<?, ?it/s]